In [1]:
import tensorflow as tf

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.metrics
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Training on C2 and Testing on C2

In [3]:
NS_case2 = pd.read_excel("../../NS_EOS_Case2.xlsx")
X_full = NS_case2.iloc[:, 0:7].values
y_full = NS_case2.iloc[:, [7,9,11]].values

from sklearn.cross_validation import train_test_split
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size = 1/6, random_state = 0)

#Standardizing the data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y = sc_y.fit_transform(y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Define the 2 HL model definition

In [7]:
def HL_2_model(i,j,result_new):
    #create model
    model = Sequential()  #creating sequential model; Dense specifies fully connected layer
    no_of_hidden_layers = 2
    node_in_each_layer = [i,j]
    model.add(Dense(node_in_each_layer[0],input_dim=7,activation='relu',kernel_initializer='normal')) #first hidden layer
    model.add(Dense(node_in_each_layer[1],activation='relu',kernel_initializer='normal')) #second hidden layer
    model.add(Dense(3,kernel_initializer='normal'))
    #compile model
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    #Train model
    train = model.fit(X,y,epochs=50,batch_size=16)  #size of dataset between 32-512
    #check performance
    y_pred = model.predict(X_test)

    y_pred = sc_y.inverse_transform(y_pred)
    #RMS value
    rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
    rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
    rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
    rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

    #R2 value
    r2 = sklearn.metrics.r2_score(y_test,y_pred)
    
    #Selection Criterion
    flag=0
    ext_dir = ''
    if rms<15 and r2>0.96:     
        flag = 1
        ext_dir = 'Good_models'
    else:
        flag=2
        ext_dir = 'Bad_models'
    result_new = result_new.append({'Model':'ANN(2HL-'+str(i)+','+str(j)+' Nodes)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                            'RMS_Lam':rms_lam, 'R2':r2, 'Flag':flag}, ignore_index=True)
    #create and open folder   
    dire = os.path.join('2HL',ext_dir,'HL_nodes-'+str(i)+'_'+str(j))
    if not os.path.exists(dire):
        os.mkdir(dire)
    plt.plot(train.history['loss'])
    plt.title("Training loss curve (50 epochs)")
    direc = os.path.join(dire,'Training loss curve (50 epochs)')
    plt.savefig(direc)
    plt.close()
    #Plotting
    titles = ['NS_Mass','NS_Rad','Lam']
    for i in range(3):
        rms = 'RMSE:'+str((np.sqrt(sklearn.metrics.mean_squared_error(y_test[:,i], y_pred[:,i]))))
        fig, ax = plt.subplots()
        ax.scatter(y_test[:,i], y_pred[:,i])
        ax.plot([y_test[:,i].min(), y_test[:,i].max()], [y_test[:,i].min(), y_test[:,i].max()], 'k--', lw=4)
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        plt.title(titles[i]+';ANN(2HL-'+str(i)+','+str(j)+' Nodes);[Case1+Case2 on Case3] \n'+rms)
        direc = os.path.join(dire,titles[i])
        plt.savefig(direc)
        plt.close()
    return result_new
    

In [8]:
result_new = pd.DataFrame(columns=['Model','RMS','RMS_NS_Mass','RMS_NS_Rad','RMS_Lam','R2','Flag'])
for i in range(1,21):
    for j in range(1,21):
        result_new = HL_2_model(i,j,result_new)
result_new

Train on 1755 samples
Epoch 1/50
1755/1755 [==============================] - 0s 200us/sample - loss: 0.9998 - mean_squared_error: 0.9998
Epoch 2/50
1755/1755 [==============================] - ETA: 0s - loss: 0.9664 - mean_squared_error: 0.96 - 0s 58us/sample - loss: 0.9710 - mean_squared_error: 0.9710
Epoch 3/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.8395 - mean_squared_error: 0.8395
Epoch 4/50
1755/1755 [==============================] - 0s 60us/sample - loss: 0.6563 - mean_squared_error: 0.6563
Epoch 5/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.5353 - mean_squared_error: 0.5353
Epoch 6/50
1755/1755 [==============================] - 0s 65us/sample - loss: 0.4657 - mean_squared_error: 0.4657
Epoch 7/50
1755/1755 [==============================] - 0s 74us/sample - loss: 0.4211 - mean_squared_error: 0.4211
Epoch 8/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.3913 - mean_squared_error: 0.3913
Epoch

1755/1755 [==============================] - 0s 57us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 16/50
1755/1755 [==============================] - 0s 57us/sample - loss: 1.0001 - mean_squared_error: 1.0001s - loss: 0.9859 - mean_squared_error: 0.98
Epoch 17/50
1755/1755 [==============================] - 0s 57us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 18/50
1755/1755 [==============================] - 0s 57us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 19/50
1755/1755 [==============================] - 0s 56us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 20/50
1755/1755 [==============================] - 0s 57us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 21/50
1755/1755 [==============================] - 0s 58us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 22/50
1755/1755 [==============================] - 0s 57us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 23/50
1755/1755 [==================

1755/1755 [==============================] - 0s 63us/sample - loss: 0.2728 - mean_squared_error: 0.2728
Epoch 35/50
1755/1755 [==============================] - 0s 65us/sample - loss: 0.2723 - mean_squared_error: 0.2723
Epoch 36/50
1755/1755 [==============================] - 0s 57us/sample - loss: 0.2714 - mean_squared_error: 0.2714
Epoch 37/50
1755/1755 [==============================] - ETA: 0s - loss: 0.2621 - mean_squared_error: 0.26 - 0s 56us/sample - loss: 0.2713 - mean_squared_error: 0.2713
Epoch 38/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.2708 - mean_squared_error: 0.2708
Epoch 39/50
1755/1755 [==============================] - 0s 65us/sample - loss: 0.2700 - mean_squared_error: 0.2700
Epoch 40/50
1755/1755 [==============================] - 0s 61us/sample - loss: 0.2697 - mean_squared_error: 0.2697
Epoch 41/50
1755/1755 [==============================] - 0s 68us/sample - loss: 0.2694 - mean_squared_error: 0.2694
Epoch 42/50
1755/1755 [=========

1755/1755 [==============================] - 0s 188us/sample - loss: 0.9930 - mean_squared_error: 0.9930
Epoch 2/50
1755/1755 [==============================] - 0s 74us/sample - loss: 0.8530 - mean_squared_error: 0.8530
Epoch 3/50
1755/1755 [==============================] - 0s 66us/sample - loss: 0.6072 - mean_squared_error: 0.6072
Epoch 4/50
1755/1755 [==============================] - 0s 64us/sample - loss: 0.4991 - mean_squared_error: 0.4991
Epoch 5/50
1755/1755 [==============================] - 0s 60us/sample - loss: 0.4444 - mean_squared_error: 0.4444
Epoch 6/50
1755/1755 [==============================] - 0s 60us/sample - loss: 0.4089 - mean_squared_error: 0.4089
Epoch 7/50
1755/1755 [==============================] - 0s 57us/sample - loss: 0.3838 - mean_squared_error: 0.3838
Epoch 8/50
1755/1755 [==============================] - 0s 58us/sample - loss: 0.3638 - mean_squared_error: 0.3638
Epoch 9/50
1755/1755 [==============================] - 0s 57us/sample - loss: 0.3486 - me

1755/1755 [==============================] - 0s 55us/sample - loss: 0.2966 - mean_squared_error: 0.2966s - loss: 0.3075 - mean_squared_error: 0.30
Epoch 19/50
1755/1755 [==============================] - 0s 55us/sample - loss: 0.2931 - mean_squared_error: 0.2931s - loss: 0.3078 - mean_squared_error: 0.30
Epoch 20/50
1755/1755 [==============================] - 0s 56us/sample - loss: 0.2901 - mean_squared_error: 0.2901
Epoch 21/50
1755/1755 [==============================] - 0s 55us/sample - loss: 0.2873 - mean_squared_error: 0.2873
Epoch 22/50
1755/1755 [==============================] - 0s 55us/sample - loss: 0.2848 - mean_squared_error: 0.2848s - loss: 0.2689 - mean_squared_error: 0.26
Epoch 23/50
1755/1755 [==============================] - ETA: 0s - loss: 0.2988 - mean_squared_error: 0.29 - 0s 55us/sample - loss: 0.2825 - mean_squared_error: 0.2825
Epoch 24/50
1755/1755 [==============================] - 0s 55us/sample - loss: 0.2806 - mean_squared_error: 0.2806
Epoch 25/50
1755/17

...

1755/1755 [==============================] - 0s 93us/sample - loss: 0.0410 - mean_squared_error: 0.0410
Epoch 41/50
1755/1755 [==============================] - 0s 74us/sample - loss: 0.0399 - mean_squared_error: 0.0399
Epoch 42/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.0400 - mean_squared_error: 0.0400
Epoch 43/50
1755/1755 [==============================] - 0s 65us/sample - loss: 0.0399 - mean_squared_error: 0.0399
Epoch 44/50
1755/1755 [==============================] - 0s 68us/sample - loss: 0.0387 - mean_squared_error: 0.0387
Epoch 45/50
1755/1755 [==============================] - 0s 65us/sample - loss: 0.0384 - mean_squared_error: 0.0384
Epoch 46/50
1755/1755 [==============================] - 0s 68us/sample - loss: 0.0388 - mean_squared_error: 0.0388
Epoch 47/50
1755/1755 [==============================] - 0s 115us/sample - loss: 0.0379 - mean_squared_error: 0.0379
Epoch 48/50
1755/1755 [==============================] - 0s 58us/sample - loss: 0.0

1755/1755 [==============================] - 0s 61us/sample - loss: 0.0836 - mean_squared_error: 0.0836
Epoch 9/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.0780 - mean_squared_error: 0.0780
Epoch 10/50
1755/1755 [==============================] - 0s 61us/sample - loss: 0.0732 - mean_squared_error: 0.0732
Epoch 11/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.0690 - mean_squared_error: 0.0690
Epoch 12/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.0675 - mean_squared_error: 0.0675
Epoch 13/50
1755/1755 [==============================] - 0s 61us/sample - loss: 0.0634 - mean_squared_error: 0.0634
Epoch 14/50
1755/1755 [==============================] - 0s 61us/sample - loss: 0.0609 - mean_squared_error: 0.0609
Epoch 15/50
1755/1755 [==============================] - 0s 61us/sample - loss: 0.0585 - mean_squared_error: 0.0585
Epoch 16/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.057

1755/1755 [==============================] - 0s 62us/sample - loss: 0.0435 - mean_squared_error: 0.0435
Epoch 28/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.0431 - mean_squared_error: 0.0431
Epoch 29/50
1755/1755 [==============================] - 0s 70us/sample - loss: 0.0421 - mean_squared_error: 0.0421
Epoch 30/50
1755/1755 [==============================] - 0s 61us/sample - loss: 0.0427 - mean_squared_error: 0.0427s - loss: 0.0358 - mean_squared_error: 0.
Epoch 31/50
1755/1755 [==============================] - 0s 64us/sample - loss: 0.0418 - mean_squared_error: 0.0418
Epoch 32/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.0404 - mean_squared_error: 0.0404
Epoch 33/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.0404 - mean_squared_error: 0.0404
Epoch 34/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.0403 - mean_squared_error: 0.0403
Epoch 35/50
1755/1755 [====================

1755/1755 [==============================] - 0s 60us/sample - loss: 0.0348 - mean_squared_error: 0.0348
Epoch 48/50
1755/1755 [==============================] - 0s 63us/sample - loss: 0.0342 - mean_squared_error: 0.0342
Epoch 49/50
1755/1755 [==============================] - 0s 62us/sample - loss: 0.0341 - mean_squared_error: 0.0341
Epoch 50/50
1755/1755 [==============================] - 0s 64us/sample - loss: 0.0336 - mean_squared_error: 0.0336


,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2,Flag
0,"ANN(2HL-1,1 Nodes)",22.892310,0.080568,0.333277,39.649161,0.739591,2
1,"ANN(2HL-1,2 Nodes)",85.818351,0.112020,0.759568,148.639761,-0.000578,2
2,"ANN(2HL-1,3 Nodes)",18.712894,0.083464,0.333109,32.409864,0.734910,2
3,"ANN(2HL-1,4 Nodes)",19.361177,0.083154,0.334861,33.532767,0.734487,2
4,"ANN(2HL-1,5 Nodes)",22.241708,0.080898,0.329293,38.522276,0.741029,2
5,"ANN(2HL-1,6 Nodes)",22.793200,0.080493,0.333149,39.477492,0.740165,2
6,"ANN(2HL-1,7 Nodes)",18.936229,0.083283,0.333582,32.796708,0.735150,2
7,"ANN(2HL-1,8 Nodes)",18.450198,0.083593,0.331662,31.954850,0.735335,2
8,"ANN(2HL-1,9 Nodes)",19.235435,0.078110,0.338250,33.314943,0.754995,2
9,"ANN(2HL-1,10 Nodes)",19.577654,0.082457,0.320591,33.907876,0.742581,2


In [13]:
#Segregating bad and good models
result_new_good = result_new[result_new['Flag']==1]
result_new_bad = result_new[result_new['Flag']==2]
result_new_v_good1 = result_new_good[result_new_good['R2']>0.975 ]  #Comment out later
result_new_v_good = result_new_v_good1[result_new_v_good1['RMS']<10]
result_new_v_good = result_new_v_good.reset_index()
result_new_v_good = result_new_v_good.drop(['index','Flag'],axis=1)
result_new_v_good

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,"ANN(2HL-14,20 Nodes)",9.299359,0.017520,0.130263,16.106425,0.978115
1,"ANN(2HL-16,14 Nodes)",9.374231,0.017832,0.138076,16.236047,0.976546
2,"ANN(2HL-17,16 Nodes)",9.773433,0.016716,0.140171,16.927493,0.976887
3,"ANN(2HL-18,15 Nodes)",9.629929,0.018567,0.139134,16.678936,0.975445
4,"ANN(2HL-18,17 Nodes)",9.748469,0.018496,0.137112,16.884277,0.975735
5,"ANN(2HL-18,20 Nodes)",9.869676,0.017511,0.138337,17.094211,0.976374
6,"ANN(2HL-19,7 Nodes)",9.935815,0.017402,0.136327,17.208787,0.976735
7,"ANN(2HL-19,14 Nodes)",9.694166,0.017856,0.133027,16.790252,0.977039
8,"ANN(2HL-19,16 Nodes)",9.456142,0.019018,0.139570,16.377913,0.975075
9,"ANN(2HL-19,18 Nodes)",9.834282,0.017204,0.138565,17.032904,0.976652


In [14]:
#Writing in file
direc1 = os.path.join('2HL','Bad_Model_details.xlsx')
result_new_bad.to_excel(direc1)
direc2 = os.path.join('2HL','Good_Model_details.xlsx')
result_new_good.to_excel(direc2)
direc3 = os.path.join('2HL','VGood_Model_details.xlsx')
result_new_v_good.to_excel(direc3)

In [16]:
list_of_V_Good_models = []
names = result_new_v_good['Model'].tolist()
for x in names:
    ij = x[8:-7].split(',')
    list_of_V_Good_models.append((int(ij[0]),int(ij[1])))
list_of_V_Good_models

[(14, 20),
 (16, 14),
 (17, 16),
 (18, 15),
 (18, 17),
 (18, 20),
 (19, 7),
 (19, 14),
 (19, 16),
 (19, 18),
 (20, 7),
 (20, 19),
 (20, 20)]

In [18]:
list_of_V_Good_models= [(14, 20),(16, 14),(17, 16),(18, 15),(18, 17),(18, 20),(19, 7),(19, 14),(19, 16),(19, 18),(20, 7),
                      (20, 19),(20, 20)]